In [5]:
import cv2
import mediapipe as mp
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import pyautogui
import random



color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        
def draw_ball_location(image, locations):
    for i in range(len(locations)-1):

        if locations[0] is None or locations[1] is None:
            continue

        cv2.line(image, tuple(locations[i]), tuple(
            locations[i+1]), color, 3)

    return image



list_ball_location = []
history_ball_locations = []

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles


screen_width, screen_height = pyautogui.size()
# print(screen_width, screen_height)


offset = 100

# For webcam input:
cap = cv2.VideoCapture(0)
# 창 생성 및 속성 설정
cv2.namedWindow("MediaPipe Hands", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("MediaPipe Hands", cv2.WND_PROP_TOPMOST, 1)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        h, w, c = image.shape

        if not success:
            print("카메라를 찾을수 없습니다.")

            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                # 엄지가 편 상태인지 확인합니다.
                thumb_finger_state = 0
                if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_height:
                    if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_height:
                            thumb_finger_state = 1

                # 검지가 편 상태인지 확인합니다.
                index_finger_state = 0
                if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * image_height:
                    if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height:
                            index_finger_state = 1

                # 중지가 편 상태인지 확인합니다.
                middle_finger_state = 0
                if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * image_height:
                    if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_height:
                            middle_finger_state = 1

                # 약지가 편 상태인지 확인합니다.
                ring_finger_state = 0
                if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y * image_height:
                    if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_height:
                            ring_finger_state = 1

                # 소지가 편 상태인지 확인합니다.
                pinky_finger_state = 0
                if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y * image_height:
                    if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y * image_height:
                            pinky_finger_state = 1

                # 왼손인지 구분
                left_hand = 0
                if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x * image_height:
                    left_hand = 1

                # 그림 부분
                use_pen = False
                if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 0 and ring_finger_state == 0 and pinky_finger_state == 0 and left_hand == 1:
                    use_pen = True

                # 지우개
                eraser_mode = False
                if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 1 and ring_finger_state == 0 and pinky_finger_state == 0 and left_hand == 1:
                    eraser_mode = True
                    
                # 색 전환
                color_change = False
                if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 1 and ring_finger_state == 1 and pinky_finger_state == 0 and left_hand == 1:
                    color_change = True

                # 전체 지우개
                use_eraser = False
                if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 1 and ring_finger_state == 1 and pinky_finger_state == 1 and left_hand == 1:
                    use_eraser = True
                

                image = Image.fromarray(image)
                draw = ImageDraw.Draw(image)

                # 마우스 이동
                if (
                    index_finger_state == 0
                    and middle_finger_state == 0
                    and ring_finger_state == 0
                    and pinky_finger_state == 0
                    and left_hand == 0
                ):
                    # 중지 밑 좌표를 사용하여 마우스를 이동시킵니다.
                    image_x = (
                        hand_landmarks.landmark[
                            mp_hands.HandLandmark.MIDDLE_FINGER_MCP
                        ].x * image_width
                    )
                    image_y = (
                        hand_landmarks.landmark[
                            mp_hands.HandLandmark.MIDDLE_FINGER_MCP
                        ].y * image_height
                    )

                    if (
                        image_x > offset
                        and image_x < image_width - offset
                        and image_y > offset
                        and image_y < image_height - offset
                    ):
                        image_x -= offset
                        image_y -= offset
                        new_image_height = image_height - offset * 2
                        new_image_width = image_width - offset * 2

                        screen_y = image_y * screen_height / new_image_height
                        screen_x = image_x * screen_width / new_image_width

                        pyautogui.moveTo(screen_x, screen_y)

                # 스크롤
                elif (
                    thumb_finger_state == 1
                    and index_finger_state == 1
                    and middle_finger_state == 1
                    and ring_finger_state == 0
                    and pinky_finger_state == 0
                    and left_hand == 0
                ):
                    # 중지 밑 좌표를 사용하여 마우스를 이동시킵니다.
                    image_x = (
                        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x
                        * image_width
                    )
                    image_y = (
                        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y
                        * image_height
                    )

                    if (
                        image_x > offset
                        and image_x < image_width - offset
                        and image_y > offset
                        and image_y < image_height - offset
                    ):
                        image_x -= offset
                        image_y -= offset
                        new_image_height = image_height - offset * 2
                        new_image_width = image_width - offset * 2

                        screen_y = image_y * screen_height / new_image_height

                        if screen_y > screen_height / 2:
                            pyautogui.scroll(-50)  # 스크롤 다운 동작
                            pre = 'scroll down'
                        else:
                            pyautogui.scroll(50)  # 스크롤 업 동작
                            pre = 'scroll up'

                # 좌클릭
                elif (
                    thumb_finger_state == 1
                    and index_finger_state == 1
                    and middle_finger_state == 0
                    and ring_finger_state == 0
                    and pinky_finger_state == 0
                    and left_hand == 0
                ):
                    pyautogui.mouseDown()
                    pyautogui.mouseUp()

                # 우클릭
                elif (
                    thumb_finger_state == 1
                    and index_finger_state == 1
                    and middle_finger_state == 1
                    and ring_finger_state == 1
                    and pinky_finger_state == 1
                    and left_hand == 0
                ):
                    pyautogui.mouseDown(button="right")
                    pyautogui.mouseUp(button="right")

                # 끄기
                elif (
                    thumb_finger_state == 1
                    and index_finger_state == 1
                    and middle_finger_state == 0
                    and ring_finger_state == 0
                    and pinky_finger_state == 1
                    and left_hand == 0
                ):
                    cv2.destroyAllWindows()
                    cap.release()



                # 색 전환
                if color_change == True:
                    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                    
                # 그림
                if use_pen == True:
                    x = int(
                        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width)
                    y = int(
                        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height)

                    list_ball_location.append((x, y))

                # 지우개 모드일 때, list_ball_location에서 좌표를 제거합니다.
                elif eraser_mode == True:
                    x = int(
                        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width)
                    y = int(
                        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_height)
                    erase_radius = 30  # 지우개로 인식할 범위 설정 (반지름 크기)

                    # list_ball_location에서 중지 끝 위치 주변의 좌표를 제거합니다.
                    for ball_location in list_ball_location:
                        ball_x, ball_y = ball_location
                        if abs(ball_x - x) <= erase_radius and abs(ball_y - y) <= erase_radius:
                            list_ball_location.remove(ball_location)


                # 전체 지우기
                elif use_eraser == True:

                    # history_ball_locations.clear()
                    list_ball_location.clear()

                image = np.array(image)

                cv2.rectangle(
                    image,
                    (offset, offset),
                    (image_width - offset, image_height - offset),
                    (255, 0, 0),
                    2,
                )

                # 손가락 뼈대를 그려줍니다.
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style(),
                )

        image = draw_ball_location(image, list_ball_location)

        for ball_locations in history_ball_locations:
            image = draw_ball_location(image, ball_locations)

        print(list_ball_location)
        cv2.imshow('MediaPipe Hands', image)

        if cv2.waitKey(1) == ord("q"):
            break

cv2.destroyAllWindows()
cap.release()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[